In [1]:
#| include: false

# Imports
import pandas as pd
import pytz
import spacy
import preprocessor as p
from emoji import demojize
from bertopic import BERTopic
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

/home/fxr/.local/share/virtualenvs/capir_transfronteriza2_2023-f1a4fPBO/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/fxr/.local/share/virtualenvs/capir_transfronteriza2_2023-f1a4fPBO/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(

In [2]:
#| include: false

!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 29.7 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
#| include: false

# read data
data = pd.read_csv('../../data/processed/tweets.csv')

/tmp/ipykernel_117286/3527960190.py:4: DtypeWarning:

Columns (6,10,18,19,20,26,28,38,39,40,45,46,47,48,54) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
#| include: false

# replace 'T' from column
data['date'] = data['local_time'].str.replace('T', ' ')

# Convert column to datetime and localize to UTC
data['date'] = pd.to_datetime(data['date'], errors='coerce').dt.tz_localize('UTC')

data.head(1)

,query,id,timestamp_utc,local_time,user_screen_name,text,possibly_sensitive,retweet_count,like_count,reply_count,...,media_types,media_alt_texts,mentioned_names,mentioned_ids,hashtags,intervention_type,intervention_text,intervention_url,country,date
0,from:TommyZambranoM,1.638175e+18,1679406309,2023-03-21T13:45:09,TommyZambranoM,Los Nacionalistas para lograr la renovación de...,0.0,30.0,117.0,58.0,...,video,NaN,pnh_oficial,201589327,librenuncamas,NaN,NaN,NaN,Honduras,2023-03-21 13:45:09+00:00


In [5]:
#| include: false

# filter data
df = data[data['user_screen_name'] == '_FamiliaEcuador']

# convert time column to Brasilia, Brazil timezone
df['date'] = df['date'].dt.tz_convert(pytz.timezone('America/Guayaquil'))

print(len(df))

2716


### Información

Los datos de este usuario cubren desde la creación de la cuenta `2018-07-28` hasta `2023-03-20`

In [6]:
#| include: false

df['date'].min()

Timestamp('2018-07-28 18:41:21-0500', tz='America/Guayaquil')

In [7]:
#| include: false

df['date'].max()

Timestamp('2023-03-20 11:57:01-0500', tz='America/Guayaquil')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2716 entries, 196700 to 199415
Data columns (total 63 columns):
 #   Column                   Non-Null Count  Dtype                            
---  ------                   --------------  -----                            
 0   query                    2716 non-null   object                           
 1   id                       2716 non-null   float64                          
 2   timestamp_utc            2716 non-null   int64                            
 3   local_time               2716 non-null   object                           
 4   user_screen_name         2716 non-null   object                           
 5   text                     2716 non-null   object                           
 6   possibly_sensitive       1663 non-null   object                           
 7   retweet_count            2716 non-null   float64                          
 8   like_count               2716 non-null   float64                          
 9   reply_

### Datos

In [9]:
# count items on column
domains_list = df['domains'].value_counts()

# return first n rows in descending order
top_domains = domains_list.nlargest(20)

top_domains

domains
youtu.be                           63
bit.ly                             43
facebook.com                       30
instagram.com                      25
youtube.com                        19
aciprensa.com                      19
eluniverso.com                     18
ecuadorporlafamilia.org            14
arquidiocesisdeguayaquil.org.ec    11
citizengo.org                      11
familiaecuador.org                 10
twitter.com                         9
open.spotify.com                    8
liveaction.org                      6
foxnews.com                         6
expreso.ec                          5
pscp.tv                             5
buff.ly                             5
drive.google.com                    5
forms.gle                           4
Name: count, dtype: int64

### Hashtags

Lista del top 20 de hashtags más usados y su frecuencia

In [10]:
# convert dataframe column to list
hashtags = df['hashtags'].to_list()

# remove nan items from list
hashtags = [x for x in hashtags if not pd.isna(x)]

# split items into a list based on a delimiter
hashtags = [x.split('|') for x in hashtags]

# flatten list of lists
hashtags = [item for sublist in hashtags for item in sublist]

# count items on list
hashtags_count = pd.Series(hashtags).value_counts()

# return first n rows in descending order
top_hashtags = hashtags_count.nlargest(20)

top_hashtags

ecuadoresprovida           307
salvemoslas2vidas          242
ecuador                    176
provida                    118
escudero                    89
abortoporviolacion          78
ecuadorporlafamilia         70
aborto                      63
coip                        58
asambleistaqueserespeta     57
leyabortistano              56
marthavillafuerte           46
escudera                    44
votoprovida2021             42
escuderos                   40
conabortonotevoto           37
prolife                     36
chantajehumanitario         35
mentirasverdes              29
juntosporlafamilia          29
Name: count, dtype: int64

### Usuarios

Top 20 de usuarios más mencionados en los tweets

In [11]:
# filter column from dataframe
users = df['mentioned_names'].to_list()

# remove nan items from list
users = [x for x in users if not pd.isna(x)]

# split items into a list based on a delimiter
users = [x.split('|') for x in users]

# flatten list of lists
users = [item for sublist in users for item in sublist]

# count items on list
users_count = pd.Series(users).value_counts()

# return first n rows in descending order
top_users = users_count.nlargest(20)

top_users

asambleaecuador    332
hectoryepezm       145
justiciaan         134
etorrescobo        132
lenin              120
lourdescuestao     105
amishijoseduco      88
agustinlaje         77
amparo_medina       71
ecuadorprovida      71
cesarrohon          61
lacristifranco      57
julietasagnay       51
gomezrobertoa       51
eluniversocom       49
marthaceciliavl     47
crisvalverdej       46
polyugarteg         44
viviana_bonilla     41
corteconstecu       38
Name: count, dtype: int64

### Likes en el tiempo

In [12]:
# plot the data using plotly
fig = px.line(df, 
              x='date', 
              y='like_count', 
              title='Likes over Time',
              template='plotly_white', 
              hover_data=['text'])

# show the plot
fig.show()

### Tokens

Lista del top 20 de los tokens más comunes y su frecuencia 

In [13]:
# load the spacy model for Spanish
nlp = spacy.load("es_core_news_sm")

# load stop words for Spanish
STOP_WORDS = nlp.Defaults.stop_words

# Function to filter stop words
def filter_stopwords(text):
    # lower text
    doc = nlp(text.lower())
    # filter tokens
    tokens = [token.text for token in doc if not token.is_stop and token.text not in STOP_WORDS and token.is_alpha]
    return ' '.join(tokens)

# apply function to dataframe column
df['text_pre'] = df['text'].apply(filter_stopwords)

# count items on column
token_counts = df["text_pre"].str.split(expand=True).stack().value_counts()[:20]

token_counts

vida                562
aborto              378
familia             354
ecuadoresprovida    316
gracias             305
ecuador             299
provida             218
apoyo               152
mujeres             139
violación           131
niños               120
nacer               119
hijos               119
concepción          118
causa               116
ley                 115
mujer               112
escudero            112
the                 108
voz                 108
Name: count, dtype: int64

### Hora

Lista de las 10 horas con más cantidad de tweets publicados

In [14]:
# extract hour from datetime column
df['hour'] = df['date'].dt.strftime('%H')

# count items on column
hours_count = df['hour'].value_counts()

# return first n rows in descending order
top_hours = hours_count.nlargest(10)

top_hours

hour
12    206
15    193
09    183
08    175
10    175
16    173
13    172
11    168
14    156
17    149
Name: count, dtype: int64

### Pataformas

Plataformas desde las que se publicaron contenidos y su frecuencia

In [15]:
df['source_name'].value_counts()

source_name
Twitter for Android    2398
Twitter Web App         151
Twitter Web Client      120
Twitter for iPhone       17
Instagram                17
TweetDeck                13
Name: count, dtype: int64

### Tópicos

Técnica de modelado de tópicos con `transformers` y `TF-IDF` 

In [16]:
#| include: false

# Remove urls, mentions, hashtags and numbers
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.NUMBER)
df['text_pre'] = df['text_pre'].apply(lambda x: p.clean(x))

# Replace emojis with descriptions
df['text_pre'] = df['text_pre'].apply(lambda x: demojize(x))

In [17]:
#| include: false

# filter column
docs = df['text_pre']

# calculate topics and probabilities
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

# training
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/85 [00:00<?, ?it/s]

2023-07-05 03:52:35,887 - BERTopic - Transformed documents to Embeddings
2023-07-05 03:52:55,133 - BERTopic - Reduced dimensionality
2023-07-05 03:52:55,506 - BERTopic - Clustered reduced embeddings


In [18]:
# visualize topics
topic_model.visualize_topics()

### Reducción de tópicos

Mapa con 10 tópicos del contenido de los tweets

In [19]:
#| include: false

# reduce the number of topics
topic_model.reduce_topics(docs, nr_topics=11)

2023-07-05 03:52:58,004 - BERTopic - Reduced number of topics from 44 to 11


In [20]:
# visualize topics
topic_model.visualize_topics()

### Términos por tópico

In [21]:
topic_model.visualize_barchart(top_n_topics=11)

### Análisis de tópicos
Selección de tópicos que tocan temas de género

In [24]:
# selection of topics
topics = [0, 2, 3]

keywords_list = []
for topic_ in topics:
    topic = topic_model.get_topic(topic_)
    keywords = [x[0] for x in topic]
    keywords_list.append(keywords)

# flatten list of lists
word_list = [item for sublist in keywords_list for item in sublist]

# use apply method with lambda function to filter rows
filtered_df = df[df['text_pre'].apply(lambda x: any(word in x for word in word_list))]

percentage = round(100 * len(filtered_df) / len(df), 2)
print(f"Del total de {len(df)} tweets de @_FamiliaEcuador, alrededor de {len(filtered_df)} hablan sobre temas de género, es decir, cerca del {percentage}%")

Del total de 2716 tweets de @_FamiliaEcuador, 1945 hablan sobre estos temas
Alrededor del 71.61% de los tweets de @_FamiliaEcuador hablan sobre estos temas


In [25]:
# drop rows with 0 values in two columns
filtered_df = filtered_df[(filtered_df.like_count != 0) & (filtered_df.retweet_count != 0)]

# add a new column with the sum of two columns
filtered_df['impressions'] = (filtered_df['like_count'] + filtered_df['retweet_count'])/2

# extract year from datetime column
filtered_df['year'] = filtered_df['date'].dt.year

# remove urls, mentions, hashtags and numbers
p.set_options(p.OPT.URL)
filtered_df['tweet_text'] = filtered_df['text'].apply(lambda x: p.clean(x))

# Create scatter plot
fig = px.scatter(filtered_df, x='like_count', 
                 y='retweet_count',
                 size='impressions', 
                 color='year',
                 hover_name='tweet_text')

# Update title and axis labels
fig.update_layout(
    title='Tweets talking about gender with most Likes and Retweets',
    xaxis_title='Number of Likes',
    yaxis_title='Number of Retweets'
)

fig.show()

### Tópicos en el tiempo

In [22]:
# convert column to list
tweets = df['text_pre'].to_list()
timestamps = df['local_time'].to_list()

topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

20it [00:00, 23.88it/s]
